In [1]:
import sqlite3
import json

In [2]:
conn = sqlite3.connect('test_assembly.db')
c = conn.cursor()

In [7]:
c.executescript('''
    CREATE TABLE IF NOT EXISTS members (
        member_id INTEGER PRIMARY KEY,
        name TEXT,
        district INTEGER,
        session_year INT
    );
    
    CREATE TABLE IF NOT EXISTS transcripts (
        date TEXT PRIMARY KEY,
        text TEXT
    );
    
    CREATE TABLE IF NOT EXISTS activity (
        activity_id INTEGER PRIMARY KEY,
        date TEXT,
        segment_id INTEGER,
        member_from TEXT,
        member_to TEXT, 
        interaction TEXT,
        FOREIGN KEY (date) REFERENCES transcripts(date)
        FOREIGN KEY (segment_id) REFERENCES transcript_segments(segment_id)
    );
    
    CREATE TABLE IF NOT EXISTS transcript_segments (
        segment_id INTEGER PRIMARY KEY,
        date TEXT,
        sequence_number INTEGER,
        member_id INTEGER,
        text TEXT,
        FOREIGN KEY (date) REFERENCES transcripts(date)
        FOREIGN KEY (member_id) REFERENCES members(member_id) 
    );
    
''')

In [3]:
# loading test case transcripts 
with open ('/Users/nickgutin/coding/DiDa-Captsone/test/test_transcipts.json', 'r') as file:
    trans = json.load(file)

In [8]:
for key in trans:
    c.execute("INSERT INTO transcripts (date, text) VALUES (?, ?)",
          (key, trans[key]) 
          )
    print(f"added raw transcript from session {key} into DB")

added raw transcript from session 6-16-25 into DB
added raw transcript from session 6-13-25 into DB
added raw transcript from session 6-12-25 into DB
added raw transcript from session 6-11-25 into DB
added raw transcript from session 6-6-25 into DB


In [4]:
import pandas as pd

test = pd.read_sql_query("SELECT * FROM transcripts", conn)

In [5]:
test.date

0    6-16-25
1    6-13-25
2    6-12-25
3    6-11-25
4     6-6-25
Name: date, dtype: object

In [19]:
#loading scraped member list
 
with open ('/Users/nickgutin/coding/DiDa-Captsone/test/members.json', 'r') as file:
    members = json.load(file)

In [20]:
members[0]

{'sessionMemberId': 2458,
 'shortName': 'ALVAREZ',
 'sessionYear': 2025,
 'districtCode': 78,
 'alternate': False,
 'memberId': 1533}

In [21]:

for mem in members: 
    
    c.execute("INSERT OR IGNORE INTO members (member_id, name, district, session_year) VALUES (?, ?, ?, ?)",
          (mem['memberId'], mem['shortName'], mem['districtCode'], mem['sessionYear']) 
          )
    
    print(f"added member {mem['shortName']} into DB")
    

added member ALVAREZ into DB
added member ANDERSON into DB
added member ANGELINO into DB
added member BAILEY into DB
added member BARCLAY into DB
added member BARRETT into DB
added member BEEPHAN into DB
added member BENDETT into DB
added member BENEDETTO into DB
added member BERGER into DB
added member BICHOTTE HERMELYN into DB
added member BLANKENBUSH into DB
added member BLUMENCRANZ into DB
added member BOLOGNA into DB
added member BORES into DB
added member BRABENEC into DB
added member BRAUNSTEIN into DB
added member BRONSON into DB
added member BROOK-KRASNY into DB
added member BROWN E into DB
added member BROWN K into DB
added member BURDICK into DB
added member BURKE into DB
added member BURROUGHS into DB
added member BUTTENSCHON into DB
added member CARROLL P into DB
added member CARROLL R into DB
added member CASHMAN into DB
added member CHANDLER-WATERMAN into DB
added member CHANG into DB
added member CHLUDZINSKI into DB
added member CLARK into DB
added member COLTON into DB

In [6]:
mem_table = pd.read_sql_query("SELECT * FROM members", conn)
mem_table.count()

member_id       151
name            151
district        151
session_year    151
dtype: int64

## activity and segment table 

### segment table

In [7]:
from chunk_scripts import *

text_list = test.text
speaker_text_test = []

count = 0
all_member_names = []
name_to_id = mem_table.set_index('name')['member_id'].to_dict()
 

for text in text_list:   
    date = test.iloc[count].date
    seq_count = 0
    
    for match in PATTERNS['speaker'].finditer(text):
        title, name, content = match.groups()
        
        normalized_name = f"{title} {name}"
        surname = normalized_name.split()[-1]
        mem_id = name_to_id.get(surname, None)
        
        speaker_text_test.append({"name": normalized_name,
                                  "member_id" : mem_id,
                                  "text" : clean_speech_text(content),
                                  "date": date,
                                  "sequence": seq_count})
        
        if not title.startswith("THE CLERK"):
            all_member_names.append(normalized_name)
        
        seq_count += 1
    
    count +=1 


In [8]:
speaker_text_test[0]

{'name': 'ACTING SPEAKER HUNTER',
 'member_id': 1105,
 'text': 'A quorum being \npresent, the Clerk will read the Journal of Sunday, June 15th.\nMrs. Peoples-Stokes.',
 'date': '6-16-25',
 'sequence': 0}

In [41]:
for segment in speaker_text_test:
    c.execute("INSERT INTO transcript_segments (date, sequence_number, member_id, text) VALUES (?, ?, ?, ?)",
          (segment['date'], segment['sequence'], segment['member_id'], segment['text']) 
          )
    
    print(f"added segment from speaker {segment['member_id']} into DB")
    

added segment from speaker 1105 into DB
added segment from speaker 591 into DB
added segment from speaker 1105 into DB
added segment from speaker 591 into DB
added segment from speaker 1105 into DB
added segment from speaker 1120 into DB
added segment from speaker 1105 into DB
added segment from speaker 1565 into DB
added segment from speaker 1105 into DB
added segment from speaker 1558 into DB
added segment from speaker 1105 into DB
added segment from speaker 670 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
added segment from speaker 1105 into DB
add

In [9]:
seg_table = pd.read_sql_query("SELECT * FROM transcript_segments", conn)
seg_table

,segment_id,date,sequence_number,member_id,text
0,1,6-16-25,0,1105.0,"A quorum being \npresent, the Clerk will read ..."
1,2,6-16-25,1,591.0,"Madam Speaker, I move \nto dispense with the f..."
2,3,6-16-25,2,1105.0,"Without objection, \nso ordered. \nMrs. People..."
3,4,6-16-25,3,591.0,"Thank you, Madam \nSpeaker. Colleagues that ar..."
4,5,6-16-25,4,1105.0,Thank you.\nWe have no housekeeping this after...
...,...,...,...,...,...
6055,6056,6-6-25,400,1141.0,"Thank you. \nOn the resolution, all those in f..."
6056,6057,6-6-25,401,673.0,"Mr. Speaker, do you -- do you have \nany furth..."
6057,6058,6-6-25,402,1141.0,"On a motion by \nMr. Weprin, page 4 on the A-C..."
6058,6059,6-6-25,403,673.0,I now move that the Assembly \nstand adjourned...


In [10]:
from chunk_scripts import *
interactions = extract_interactions(speaker_text_test)
interactions

[{'from_member_id': 1127,
  'from_member_name': 'MS. WALSH',
  'to_member_id': 633,
  'to_member_name': 'MR. BRONSON',
  'interaction_type': 'question',
  'text_snippet': 'Will the \nsponsor yield',
  'date': '6-16-25',
  'sequence': 33},
 {'from_member_id': 633,
  'from_member_name': 'MR. BRONSON',
  'to_member_id': 1127,
  'to_member_name': 'MS. WALSH',
  'interaction_type': 'address',
  'text_snippet': 'MS. WALSH',
  'date': '6-16-25',
  'sequence': 58},
 {'from_member_id': 1465,
  'from_member_name': 'MR. GANDOLFO',
  'to_member_id': 665,
  'to_member_name': 'MR. OTIS',
  'interaction_type': 'address',
  'text_snippet': 'MR. OTIS',
  'date': '6-16-25',
  'sequence': 94},
 {'from_member_id': 1465,
  'from_member_name': 'MR. GANDOLFO',
  'to_member_id': 665,
  'to_member_name': 'MR. OTIS',
  'interaction_type': 'address',
  'text_snippet': 'Mr. Otis',
  'date': '6-16-25',
  'sequence': 107},
 {'from_member_id': 1511,
  'from_member_name': 'MR. BLUMENCRANZ',
  'to_member_id': 665,
  '

In [11]:
# Extract interactions
interactions = extract_interactions(speaker_text_test)

# Get segment_id mapping from database
c.execute("SELECT segment_id, date, sequence_number FROM transcript_segments")
segment_lookup = {(row[1], row[2]): row[0] for row in c.fetchall()}

# Prepare data for insertion
insert_data = []
for interaction in interactions:
    date = interaction['date']
    sequence = interaction['sequence']
    
    # Look up the segment_id
    segment_id = segment_lookup.get((date, sequence))
    
    if segment_id is None:
        print(f"Warning: No segment_id found for date={date}, sequence={sequence}")
        continue
    
    insert_data.append((
        date,
        segment_id,
        interaction['from_member_id'],
        interaction['to_member_id'],
        interaction['interaction_type']
    ))

# Bulk insert
c.executemany(
    """
    INSERT INTO activity (date, segment_id, member_from, member_to, interaction)
    VALUES (?, ?, ?, ?, ?)
    """,
    insert_data
)

conn.commit()
print(f"Inserted {len(insert_data)} interactions into activity table")

Inserted 376 interactions into activity table
